# Brand Network Extraction from PostgresSQL DB

Please ensure to set up table and views before running the below.

Input: 

- good_brands_list from 4_Good_Brand_Sampling/all_good_brands.csv
- bad_brands_list from prior research

Output: Files are too large to enclose here, please follow the instructions below to generate
- Edgelist CSVs by brand
- Nodelist CSVs by brand

In [1]:
import pandas as pd
import numpy as np
import psycopg2

In [2]:
good = pd.read_csv('4_Good_Brand_Sampling/all_good_brands.csv')
good_brands_list = list(good['brand'])

In [3]:
print(good_brands_list)

['Belkin', 'Comsoon', 'DOSS', 'JVC', 'Native Union', 'Poweradd', 'Skullcandy', 'StarTech', 'UGREEN', 'mophie', 'iFlash', 'QualGear', 'CableJive', 'Mopar', 'Optimal Shop', 'COOLEAD', 'VidPro', 'MFJ', 'KINESIS', 'CalDigit', 'EZOPower', 'nextec', 'Lenovo', 'Jabees', 'Loctek', 'DXG', 'Impecca', 'Tera Grand', '1MORE', 'Lenspen', 'ABLEGRID', 'Tzumi', 'Reshow', 'VAKOO', 'Optoma', 'BENEWY', 'Gechic', 'BUBM', 'TYT', 'Avantree', 'Zacro', 'YETOR', 'Suptek', 'JEXON', 'JOOAN', 'CableGeeker', 'ORICO', 'Enermax', 'Vackoey', 'ANCwear', 'WEme', 'Seagate', 'Crucial', 'Cisco', 'TENVIS', 'Powerextra', 'Yamaha Audio', 'Elsse', 'Kingston', 'Tranesca', 'Syncwire', 'GMYLE', 'CHUWI', 'MOHOO', 'GW Security Inc', 'Golf Buddy', 'High Sierra', 'Cerwin-Vega', 'iPazzPort', 'Evercool', 'HyperX', 'Koolertron', 'Ritz Camera', 'Tensun', 'Lowpricenice', 'NeuTab', 'DEEGO', 'LiteFuze', 'FITUEYES', 'Naxa Electronics', 'Power Acoustik', 'VIVO', 'Macally', 'AudioQuest', 'EloBeth', 'Supersonic', 'Sto-Fen', 'Mount Factory', 'RC

In [3]:
# Too small: Seneo, Homasy, Sunvalley
bad_brands_list = ['TaoTronics', 'Mpow', 'VicTsing', 'HooToo', 'AUKEY', 'Vtin', 'RAVPower', 'CHOETECH', 'TOMTOP', 'YKS',
                  'VOGEK', 'LETSCOM', 'Amzdeal', 'OMORC']

## 1. Extract Network from 2015 to 2017

In [14]:
network_columns = ['common_item',
                   'brand', 
                   'banned',
                   'user_i',
                   'user_i_rating',
                   'user_i_sentiment',
                   'user_i_ratingdate',
                   'user_j',
                   'user_j_rating',
                   'user_j_sentiment',
                   'user_j_ratingdate',
                   'rating_daydiff']

network_query = "SELECT a.item_id as common_item, a.brand, a.banned, "\
                    "a.user_id as user_i, a.rating as user_i_rating, a.sentiment as user_i_sentiment, a.date as user_i_ratingdate, "\
                    "b.user_id as user_j, b.rating as user_j_rating, b.sentiment as user_j_sentiment, b.date as user_j_ratingdate, "\
                    "DATE_PART('day', a.date::timestamp - b.date::timestamp) as rating_daydiff "\
                "FROM analysis_table a, analysis_table b "\
                "WHERE a.item_id = b.item_id "\
                    "AND a.brand = b.brand "\
                    "AND a.brand = {} "\
                    "AND a.user_id != b.user_id "\
                    "AND a.sentiment = b.sentiment "\
                    "AND DATE_PART('day', a.date::timestamp - b.date::timestamp) < 90 "\
                    "AND DATE_PART('day', a.date::timestamp - b.date::timestamp) > -1 "\
                    "AND DATE_PART('year', a.date::timestamp) >= 2015 "\
                    "AND DATE_PART('year', a.date::timestamp) < 2018 "\
                    "AND DATE_PART('year', b.date::timestamp) >= 2015 "\
                    "AND DATE_PART('year', b.date::timestamp) < 2018 "

try:
    for name in bad_brands_list:
        connection = psycopg2.connect(user="YOUR_DB_USERNAME",
                                      password="YOUR_DB_PASSWORD",
                                      host="127.0.0.1",
                                      port="5432",
                                      database="amazon")
        cursor = connection.cursor()
        postgreSQL_select_Query = network_query.format('\''+name+'\'')

        cursor.execute(postgreSQL_select_Query)
        print("Selecting rows from mobile table using cursor.fetchall")
        records = cursor.fetchall()

        df_export = pd.DataFrame(records)
        df_export.columns = network_columns
        df_export.to_csv('{}_2015to2017.csv'.format(name))

except (Exception, psycopg2.Error) as error:
    print("Error while fetching data from PostgreSQL", error)

finally:
    # closing database connection.
    if connection:
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")

Selecting rows from mobile table using cursor.fetchall
Selecting rows from mobile table using cursor.fetchall
Selecting rows from mobile table using cursor.fetchall
Selecting rows from mobile table using cursor.fetchall
PostgreSQL connection is closed


In [50]:
df_export.head()

,common_item,brand,banned,user_i,user_i_rating,user_i_sentiment,user_i_ratingdate,user_j,user_j_rating,user_j_sentiment,user_j_ratingdate,rating_daydiff
0,B000026D8E,Seagate,0,A25YJ9QZQ3K5IB,1.0,negative,2016-02-13,A6XDZ1MJ0B1A2,1.0,negative,2016-01-19,25.0
1,B000026D8E,Seagate,0,A2O66U5STZJOID,5.0,positive,2015-03-18,A30ERAD1AV6B6E,5.0,positive,2015-01-04,73.0
2,B000026D8E,Seagate,0,A2YXTH0O36P9WY,5.0,positive,2015-12-28,A8AXLBARJ5XK7,5.0,positive,2015-11-06,52.0
3,B000026D8E,Seagate,0,A8AXLBARJ5XK7,5.0,positive,2015-11-06,A11L6YFBNLYFG0,5.0,positive,2015-08-23,75.0
4,B000026D8E,Seagate,0,A8AXLBARJ5XK7,5.0,positive,2015-11-06,AV4ZMWMUALCB6,5.0,positive,2015-09-16,51.0


## 2. Extract Node List (2015 to 2017)

In [18]:
network_columns = ['user_id',
                   'date', 
                   'item_id',
                   'rating',
                   'brand',
                   'sentiment',
                   'banned']

network_query = "SELECT user_id, date, item_id, rating, brand, sentiment, banned "\
                "FROM analysis_table "\
                "WHERE brand = {} "\
                    "AND DATE_PART('year', date::timestamp) >= 2015 "\
                    "AND DATE_PART('year', date::timestamp) < 2018 "

In [21]:
try:
    for name in bad_brands_list:
        connection = psycopg2.connect(user="YOUR_DB_USERNAME",
                                      password="YOUR_DB_PASSWORD",
                                      host="127.0.0.1",
                                      port="5432",
                                      database="amazon")
        cursor = connection.cursor()
        postgreSQL_select_Query = network_query.format('\''+name+'\'')

        cursor.execute(postgreSQL_select_Query)
        print("Selecting rows from mobile table using cursor.fetchall")
        records = cursor.fetchall()

        df_export = pd.DataFrame(records)
        df_export.columns = network_columns
        df_export.to_csv('{}_2015to2017_nodelist.csv'.format(name))

except (Exception, psycopg2.Error) as error:
    print("Error while fetching data from PostgreSQL", error)

finally:
    # closing database connection.
    if connection:
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")

Selecting rows from mobile table using cursor.fetchall
Selecting rows from mobile table using cursor.fetchall
Selecting rows from mobile table using cursor.fetchall
Selecting rows from mobile table using cursor.fetchall
Selecting rows from mobile table using cursor.fetchall
Selecting rows from mobile table using cursor.fetchall
Selecting rows from mobile table using cursor.fetchall
Selecting rows from mobile table using cursor.fetchall
Selecting rows from mobile table using cursor.fetchall
Selecting rows from mobile table using cursor.fetchall
Selecting rows from mobile table using cursor.fetchall
Selecting rows from mobile table using cursor.fetchall
Selecting rows from mobile table using cursor.fetchall
Selecting rows from mobile table using cursor.fetchall
PostgreSQL connection is closed
